In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pytest
from unittest.mock import patch, MagicMock
from core.common.notifications import NotificationManager

## Test Initialization

In [ ]:
def test_notification_manager_init():
    """Test NotificationManager initialization"""
    manager = NotificationManager(app_name="TestApp")
    
    assert manager.app_name == "TestApp"
    assert hasattr(manager, 'platform')
    print(f"✅ Initialization test passed (platform: {manager.platform})")

test_notification_manager_init()

## Test Platform Detection

In [ ]:
def test_platform_detection():
    """Test platform detection"""
    manager = NotificationManager()
    
    assert manager.platform in ['Windows', 'Linux', 'Darwin', 'unknown']
    print(f"✅ Platform detection test passed: {manager.platform}")

test_platform_detection()

## Test Basic Notification

In [ ]:
def test_basic_notification():
    """Test basic notification (will fallback to print if no notification system)"""
    manager = NotificationManager()
    
    # Should not raise exception
    try:
        manager.notify("Test Title", "Test Message", duration=1)
        print("✅ Basic notification test passed")
    except Exception as e:
        pytest.fail(f"Notification failed: {e}")

test_basic_notification()

## Test Alert Methods

In [ ]:
def test_battery_alert():
    """Test battery alert notification"""
    manager = NotificationManager()
    
    try:
        manager.battery_alert(15.0)
        print("✅ Battery alert test passed")
    except Exception as e:
        pytest.fail(f"Battery alert failed: {e}")

test_battery_alert()

In [ ]:
def test_cpu_alert():
    """Test CPU alert notification"""
    manager = NotificationManager()
    
    try:
        manager.cpu_alert(95.0)
        print("✅ CPU alert test passed")
    except Exception as e:
        pytest.fail(f"CPU alert failed: {e}")

test_cpu_alert()

In [ ]:
def test_memory_alert():
    """Test memory alert notification"""
    manager = NotificationManager()
    
    try:
        manager.memory_alert(92.0, available_gb=1.5)
        print("✅ Memory alert test passed")
    except Exception as e:
        pytest.fail(f"Memory alert failed: {e}")

test_memory_alert()

In [ ]:
def test_disk_alert():
    """Test disk alert notification"""
    manager = NotificationManager()
    
    try:
        manager.disk_alert('C:', 88.0)
        print("✅ Disk alert test passed")
    except Exception as e:
        pytest.fail(f"Disk alert failed: {e}")

test_disk_alert()

## Test Windows Notifications (Mock)

In [ ]:
@patch('platform.system', return_value='Windows')
@patch('core.common.notifications.WINDOWS_TOAST_AVAILABLE', True)
def test_windows_notification_mock(mock_platform):
    """Test Windows notification with mocking"""
    try:
        from win10toast import ToastNotifier
        
        with patch.object(ToastNotifier, 'show_toast') as mock_toast:
            manager = NotificationManager()
            manager.notify("Test", "Message")
            
            # Verify show_toast was called
            assert mock_toast.called or manager.platform == 'Windows'
            print("✅ Windows notification mock test passed")
    except ImportError:
        print("⚠️ Windows notification test skipped (win10toast not available)")

test_windows_notification_mock()

## Test Notification Throttling

In [ ]:
def test_multiple_notifications():
    """Test sending multiple notifications"""
    manager = NotificationManager()
    
    # Send multiple notifications (should not fail)
    try:
        for i in range(3):
            manager.notify(f"Test {i}", f"Message {i}", duration=1)
        print("✅ Multiple notifications test passed")
    except Exception as e:
        pytest.fail(f"Multiple notifications failed: {e}")

test_multiple_notifications()

## Test Fallback Mechanism

In [ ]:
@patch('core.common.notifications.WINDOWS_TOAST_AVAILABLE', False)
@patch('subprocess.run', side_effect=FileNotFoundError)
def test_fallback_notification(mock_subprocess):
    """Test fallback to print when notification systems unavailable"""
    manager = NotificationManager()
    
    # Should fallback to print without raising exception
    try:
        with patch('builtins.print') as mock_print:
            manager.notify("Test", "Fallback test")
            # On some platforms, print fallback is used
            assert True  # Just verify no exception
        print("✅ Fallback notification test passed")
    except Exception as e:
        pytest.fail(f"Fallback failed: {e}")

test_fallback_notification()

## Run All Tests

In [ ]:
# Run all tests
pytest.main(['-v', '--tb=short', __file__])